In [1]:
import numpy as np

sequence_length = 10
frame_height, frame_width, channels = 224, 224, 3
num_classes = 2

X_data = np.memmap(r'E:\PosePerfect\Dataset Creation\X_final.dat', dtype='float32', mode='r', shape=(5082, sequence_length, frame_height, frame_width, channels))
y_data = np.memmap(r'E:\PosePerfect\Dataset Creation\y_final.dat', dtype='int32', mode='r', shape=(5082,))

In [4]:
# Build
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout

def build_cnn_lstm_model(sequence_length, height, width, channels, num_classes):
    model = Sequential([
        # CNN: Process each frame individually using TimeDistributed
        TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'), 
                        input_shape=(sequence_length, height, width, channels)),
        TimeDistributed(MaxPooling2D(pool_size=(2, 2))),
        TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same')),
        TimeDistributed(MaxPooling2D(pool_size=(2, 2))),
        TimeDistributed(Flatten()),
        
        # LSTM: Learn temporal relationships between frames
        LSTM(128, return_sequences=True),
        Dropout(0.5),
        LSTM(64),
        Dropout(0.5),
        
        # Dense layers for classification
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

cnn_lstm_model = build_cnn_lstm_model(sequence_length, frame_height, frame_width, channels, num_classes)
cnn_lstm_model.summary()


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

print(f"Training samples: {X_train.shape[0]}, Validation samples: {X_val.shape[0]}")


In [6]:
# Train
history = cnn_lstm_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=16,
    verbose=1
)


In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

# Parameters
sequence_length = 10
frame_height = 224
frame_width = 224
channels = 3
num_classes = 2  # Number of exercise types: biceps curl, squats
batch_size = 5

# Paths to memory-mapped files
X_path = r'E:\PosePerfect\Dataset Creation\X_final.dat'
y_path = r'E:\PosePerfect\Dataset Creation\y_final.dat'

# Dataset shape
total_sequences = 5082  # Adjust based on your dataset
X_shape = (total_sequences, sequence_length, frame_height, frame_width, channels)
y_shape = (total_sequences,)

# **Step 1: Split the data into training and validation indices**
indices = np.arange(total_sequences)  # Create an array of indices
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)  # Split indices

print(f"Training samples: {len(train_indices)}, Validation samples: {len(val_indices)}")

# **Step 2: Data generator**
def data_generator(X_path, y_path, indices, batch_size):
    # Load memory-mapped arrays
    X_data = np.memmap(X_path, dtype='float32', mode='r', shape=X_shape)
    y_data = np.memmap(y_path, dtype='int32', mode='r', shape=y_shape)
    num_samples = len(indices)
    
    while True:
        np.random.shuffle(indices)  # Shuffle indices for every epoch
        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i + batch_size]
            X_batch = X_data[batch_indices]
            y_batch = y_data[batch_indices]
            
            # One-hot encode y_batch for categorical crossentropy
            y_batch = np.eye(num_classes)[y_batch]  # One-hot encoding
            
            yield X_batch, y_batch

# **Step 3: Build the CNN-LSTM model**
def build_cnn_lstm_model(sequence_length, height, width, channels, num_classes):
    model = Sequential([
        # CNN: Process each frame individually using TimeDistributed
        TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'), 
                        input_shape=(sequence_length, height, width, channels)),
        TimeDistributed(MaxPooling2D(pool_size=(2, 2))),
        TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same')),
        TimeDistributed(MaxPooling2D(pool_size=(2, 2))),
        TimeDistributed(Conv2D(128, (3, 3), activation='relu', padding='same')),
        TimeDistributed(MaxPooling2D(pool_size=(2, 2))),
        TimeDistributed(Flatten()),  # Flatten the output of the CNN

        # LSTM: Learn temporal relationships between frames
        LSTM(128, return_sequences=True),
        Dropout(0.5),
        LSTM(64),
        Dropout(0.5),

        # Dense layers for classification
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Instantiate the model
cnn_lstm_model = build_cnn_lstm_model(sequence_length, frame_height, frame_width, channels, num_classes)
cnn_lstm_model.summary()

# **Step 4: Train the model using data generators**
train_generator = data_generator(X_path, y_path, train_indices, batch_size)
val_generator = data_generator(X_path, y_path, val_indices, batch_size)

# Calculate steps per epoch
steps_per_epoch_train = len(train_indices) // batch_size
steps_per_epoch_val = len(val_indices) // batch_size

# Train the model
cnn_lstm_model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    validation_data=val_generator,
    validation_steps=steps_per_epoch_val,
    epochs=10  # Adjust epochs as needed
)


In [4]:
model_save_path = r'/Model/SavedModels/cnn_lstm_model_try1.h5'
cnn_lstm_model.save(model_save_path)
print(f"Model saved at: {model_save_path}")

In [1]:
from tensorflow.keras.models import load_model

loaded_model = load_model(r'E:\PosePerfect\Model\SavedModels\cnn_lstm_model_try1.h5')
loaded_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, 10, 224, 224,   │           896 │
│ (TimeDistributed)               │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 10, 112, 112,   │             0 │
│ (TimeDistributed)               │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 10, 112, 112,   │        18,496 │
│ (TimeDistributed)               │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 10, 56, 56, 64) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 10, 56, 56,     │        73,856 │
│ (TimeDistributed)               │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 10, 28, 28,     │             0 │
│ (TimeDistributed)               │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_6              │ (None, 10, 100352)     │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 10, 128)        │    51,446,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,591,076 (196.80 MB)

 Trainable params: 51,591,074 (196.80 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [4]:
example_input = X_data = np.memmap(r'E:\PosePerfect\Model\Process Example\X_exam.dat', dtype='float32', mode='r', shape=(40, 10, 224, 224, 3))

predictions = loaded_model.predict(example_input)
predicted_class = np.argmax(predictions, axis=1)

print(f"Predicted class: {predicted_class}")

NameError: name 'loaded_model' is not defined

In [11]:
print('heyyy')

SyntaxError: invalid syntax (2691026085.py, line 1)